<a href="https://colab.research.google.com/github/1kaiser/test2023/blob/main/OpenAltimetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenAltimetry ICESat-2: Elevation Workflow
Overview


This notebook pulls data from ICESat-2 surface-specific products - ATL06, ATL07, ATL08, ATL10, ATL12, ATL13 via the OpenAltimetry API.


Instructions
Replace the "OA_API_URL" string with the data copied from : "Copy API URL" from the OpenAltimetry elevation chart and start visualizing your data!
Author: Minh Phan - mnphan@ucsd.edu. Based off the work of Luis Lopez - luis.lopez@nsidc.org

In [ ]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot

# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl06?date=2019-09-06&minx=65.77529174029638&miny=64.53861178731293&maxx=66.28615599810888&maxy=65.00003756856293&trackId=1080&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

OA_API_URL += '&client=jupyter';

OA_BEAMS = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']

df_beams = []
oa_plots = []

# This function will request the 6 tracks using OpenAltimetry's API
def getData():
    series = []
    b_url = OA_API_URL

    print('Requesting data from OA')
    r = requests.get(b_url)
    data = r.json()
    return data


elevation_data = getData()
product = elevation_data["product"]

print('Reading data product: ' + product)

for series_beam in elevation_data['series']:
    if any(word in series_beam['beam'] for word in OA_BEAMS):
        series = []
        
        data_name = 'lat_lon_elev'
        if product == 'ATL08':
            data_name = 'lat_lon_elev_canopy'

        for p in series_beam[data_name]:
            series.append({
                'lat': p[0],
                'lon': p[1],
                'h': p[2]
            })
            
        print ('beam ' + series_beam['beam'] + ' - len = ' + str(len(series)))

        if (len(series) > 0):
            df = pd.DataFrame.from_dict(series)
            df.name = series_beam['beam']
            df_beams.append(df)
            
print('df_beams len: ' + str(len(df_beams)))

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'],
                                    marker=dict(
                                        size=2,
                                        color=df['h'],
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ),
                                    line=dict(
                                        color='darkblue',
                                        width=2
                                    )
                                )
                   )

layout = go.Layout(
    width=1200,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

print('Plotting...')

fig = go.Figure(data=oa_plots, layout=layout)

iplot(fig)


https://openaltimetry.org

In [ ]:
curl -X GET "https://openaltimetry.org/data/api/icesat2/getTracks?date=2020-05-13&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&outputFormat=csv" -H "accept: */*"